In [64]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [66]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [67]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [68]:
x = df.drop('Outcome', axis=1).values
y = df['Outcome'].values

In [69]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [70]:
x = scaler.fit_transform(x)

In [71]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [72]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [73]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [74]:
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5630 - loss: 0.7337 - val_accuracy: 0.6753 - val_loss: 0.6281
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6238 - loss: 0.6555 - val_accuracy: 0.7143 - val_loss: 0.5850
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6982 - loss: 0.6030 - val_accuracy: 0.7403 - val_loss: 0.5573
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7425 - loss: 0.5692 - val_accuracy: 0.7857 - val_loss: 0.5374
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7755 - loss: 0.5197 - val_accuracy: 0.7727 - val_loss: 0.5215
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7472 - loss: 0.5368 - val_accuracy: 0.7857 - val_loss: 0.5135
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7704 - loss: 0.4947 - val_accuracy: 0.7792 - val_loss: 0.5054
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7663 - loss: 0.5141 - val_accuracy: 0.7792 - v

In [75]:
# 1. how to select appropriate optimizer
# no. of nodes in hidden layer
# how to select no. of hidden layers
# all in all one model

In [76]:
%pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [77]:
import kerastuner as kt

In [78]:
def build_model(hp):

    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adagrad', 'adadelta']), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [79]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

In [80]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.48051947355270386

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 05s


In [81]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [82]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [83]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [84]:
model.fit(x_train, y_train, batch_size=32, epochs=100,initial_epoch=11, validation_data=(x_test, y_test))

Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7793 - loss: 0.4845 - val_accuracy: 0.7727 - val_loss: 0.5089
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7631 - loss: 0.4811 - val_accuracy: 0.7468 - val_loss: 0.5011
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7834 - loss: 0.4643 - val_accuracy: 0.7468 - val_loss: 0.4980
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7833 - loss: 0.4606 - val_accuracy: 0.7662 - val_loss: 0.4934
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7815 - loss: 0.4589 - val_accuracy: 0.7597 - val_loss: 0.4924
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7655 - loss: 0.4874 - val_accuracy: 0.7597 - val_loss: 0.4913
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7957 - loss: 0.4529 - val_accuracy: 0.7532 - val_loss: 0.4897
Epoch 19/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7815 - loss: 0.4527 - val_accuracy: 0.

In [85]:
def build_model(hp):

    model = Sequential()
    
    units = hp.Int('units', min_value=8, max_value=512)

    model.add(Dense(units = units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [86]:
def build_model(hp):

    model = Sequential()
    
    units = hp.Int('units', min_value=8, max_value=512)

    model.add(Dense(units = units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [87]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 05s


In [88]:
tuner.get_best_hyperparameters()[0].values

{'units': 264}

In [89]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [90]:
model.fit(x_train, y_train, batch_size=32, epochs=100,initial_epoch=11, validation_data=(x_test, y_test))

Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7833 - loss: 0.4572 - val_accuracy: 0.7662 - val_loss: 0.4965
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7805 - loss: 0.4454 - val_accuracy: 0.7727 - val_loss: 0.4943
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8103 - loss: 0.4073 - val_accuracy: 0.7792 - val_loss: 0.4993
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7780 - loss: 0.4406 - val_accuracy: 0.7532 - val_loss: 0.5044
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7879 - loss: 0.4342 - val_accuracy: 0.7597 - val_loss: 0.5025
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7725 - loss: 0.4524 - val_accuracy: 0.7597 - val_loss: 0.5066
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7808 - loss: 0.4481 - val_accuracy: 0.7597 - val_loss: 0.5068
Epoch 19/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7797 - loss: 0.4346 - val_accuracy: 0.

In [91]:
def build_model(hp):

    model = Sequential()

    model.add(Dense(72, activation='relu', input_dim=8))

    for i in range(hp.Int('num_layers', mun_value = 1, max_value = 10)):
        model.add(Dense(72, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [93]:
def build_model(hp):

    model = Sequential()

    model.add(Dense(72, activation='relu', input_dim=8))

    for i in range(hp.Int('num_layers', mun_value = 1, max_value = 10)):
        model.add(Dense(72, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model

Reloading Tuner from output\diabetes\tuner0.json


In [94]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

In [95]:
tuner.get_best_hyperparameters()[0].values

{'units': 210}